In [73]:
import sqlite3
import pandas as pd
import json
import requests
import os

## Création BDD et tables

In [74]:
try:
    db = sqlite3.connect("foil.db")
    
    cursor = db.cursor()
    
    cursor.execute('''create table stations (       
    
    id_station integer primary key,
    name varchar,
    latitude float,
    longitude float)''')
    
    cursor.execute('''create table mesures (       
    
    id_mesure integer primary key autoincrement unique,
    date varchar,
    id_station integer,
    pressure float,
    wind_heading float,
    wind_speed_avg float,
    wind_speed_max float,
    wind_speed_min float,
    status_state varchar,
    foreign key (id_station) references stations(id_station))''')
                                                     
    db.commit()
    
    cursor.close()
    db.close()
except:
    print("Une erreur est survenue lors de la creation de la base")
    exit(1)
    
print("La base de données a été créée")  

La base de données a été créée


## Création liste Stations

In [75]:
data_station_113=[]
response = requests.get('http://api.pioupiou.fr/v1/live/113')
station_113 = json.loads(response.text)
data_station_113.append([station_113['data']['id'],
                        station_113['data']['meta']['name'],
                        station_113['data']['location']['latitude'],
                        station_113['data']['location']['longitude']])
data_station_113

[[113, 'Brétignolles sur mer - La Normandelière', 46.613775, -1.861274]]

In [76]:
data_station_307=[]
response = requests.get('http://api.pioupiou.fr/v1/live/307')
station_307 = json.loads(response.text)
data_station_307.append([station_307['data']['id'],
                        station_307['data']['meta']['name'],
                        station_307['data']['location']['latitude'],
                        station_307['data']['location']['longitude']])
data_station_307

[[307, 'Pioupiou 307', 46.450667, -1.581897]]

In [77]:
data_station_308=[]
response = requests.get('http://api.pioupiou.fr/v1/live/308')
station_308 = json.loads(response.text)
data_station_308.append([station_308['data']['id'],
                        station_308['data']['meta']['name'],
                        station_308['data']['location']['latitude'],
                        station_308['data']['location']['longitude']])
data_station_308

[[308, 'Pioupiou 308', 46.692344, -1.764994]]

## Création liste Mesures

In [78]:
data_mesure_113=[]
response = requests.get('http://api.pioupiou.fr/v1/live/113')
mesure_113 = json.loads(response.text)
data_mesure_113.append([mesure_113['data']['measurements']['date'],
                        mesure_113['data']['id'],
                        mesure_113['data']['measurements']['pressure'],
                        mesure_113['data']['measurements']['wind_heading'],
                        mesure_113['data']['measurements']['wind_speed_avg'],
                        mesure_113['data']['measurements']['wind_speed_max'],
                        mesure_113['data']['measurements']['wind_speed_min'],
                        mesure_113['data']['status']['state']])
data_mesure_113

[['2021-03-11T10:57:15.000Z', 113, None, 225, 31, None, None, 'on']]

In [79]:
data_mesure_307=[]
response = requests.get('http://api.pioupiou.fr/v1/live/307')
mesure_307 = json.loads(response.text)
data_mesure_307.append([mesure_307['data']['measurements']['date'],
                        mesure_307['data']['id'],
                        mesure_307['data']['measurements']['pressure'],
                        mesure_307['data']['measurements']['wind_heading'],
                        mesure_307['data']['measurements']['wind_speed_avg'],
                        mesure_307['data']['measurements']['wind_speed_max'],
                        mesure_307['data']['measurements']['wind_speed_min'],
                        mesure_307['data']['status']['state']])
data_mesure_307

[['2021-03-11T10:56:51.000Z', 307, None, 247.5, 25.5, 35.5, 14, 'on']]

In [80]:
data_mesure_308=[]
response = requests.get('http://api.pioupiou.fr/v1/live/308')
mesure_308 = json.loads(response.text)
data_mesure_308.append([mesure_308['data']['measurements']['date'],
                        mesure_308['data']['id'],
                        mesure_308['data']['measurements']['pressure'],
                        mesure_308['data']['measurements']['wind_heading'],
                        mesure_308['data']['measurements']['wind_speed_avg'],
                        mesure_308['data']['measurements']['wind_speed_max'],
                        mesure_308['data']['measurements']['wind_speed_min'],
                        mesure_308['data']['status']['state']])
data_mesure_308

[['2021-03-11T10:50:46.000Z', 308, None, 90, 0, 0, 0, 'on']]

## Connexion BDD

In [81]:
conn = sqlite3.connect('foil.db')

## Insertion données

In [82]:
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

In [84]:
sql_inserer_station = """
    INSERT INTO stations
    (id_station, name, latitude, longitude)
    VALUES (?, ?, ?, ?)
"""

In [85]:
inserer_donnees(conn, sql_inserer_station, data_station_113)

Les données ont été insérées avec succès


In [86]:
inserer_donnees(conn, sql_inserer_station, data_station_308)

Les données ont été insérées avec succès


In [87]:
inserer_donnees(conn, sql_inserer_station, data_station_307)

Les données ont été insérées avec succès


In [89]:
sql_inserer_mesure = """
    INSERT INTO mesures
    (date, id_station, pressure, wind_heading, wind_speed_avg, wind_speed_max, wind_speed_min, status_state)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

In [90]:
inserer_donnees(conn, sql_inserer_mesure, data_mesure_113)

Les données ont été insérées avec succès


In [91]:
inserer_donnees(conn, sql_inserer_mesure, data_mesure_307)

Les données ont été insérées avec succès


In [92]:
inserer_donnees(conn, sql_inserer_mesure, data_mesure_308)

Les données ont été insérées avec succès


In [88]:
sql = '''
SELECT *
FROM stations'''

sql = pd.read_sql_query(sql, conn)
sql

,id_station,name,latitude,longitude
0,113,Brétignolles sur mer - La Normandelière,46.613775,-1.861274
1,307,Pioupiou 307,46.450667,-1.581897
2,308,Pioupiou 308,46.692344,-1.764994


In [93]:
sql = '''
SELECT *
FROM mesures'''

sql = pd.read_sql_query(sql, conn)
sql

,id_mesure,date,id_station,pressure,wind_heading,wind_speed_avg,wind_speed_max,wind_speed_min,status_state
0,1,2021-03-11T10:57:15.000Z,113,None,225.0,31.0,NaN,NaN,on
1,2,2021-03-11T10:56:51.000Z,307,None,247.5,25.5,35.5,14.0,on
2,3,2021-03-11T10:50:46.000Z,308,None,90.0,0.0,0.0,0.0,on


In [113]:
liste_stations = [113, 307, 308]

In [114]:
data_stations=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_stations.append([response_info['data']['id'], response_info['data']['meta']['name'], response_info['data']['location']['latitude'], response_info['data']['location']['longitude']])

In [115]:
data_stations

[[113, 'Brétignolles sur mer - La Normandelière', 46.613775, -1.861274],
 [307, 'Pioupiou 307', 46.450667, -1.581897],
 [308, 'Pioupiou 308', 46.692344, -1.764994]]

In [116]:
data_mesures=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_mesures.append([response_info['data']['measurements']['date'], response_info['data']['id'], response_info['data']['measurements']['pressure'], response_info['data']['measurements']['wind_heading'], response_info['data']['measurements']['wind_speed_avg'], response_info['data']['measurements']['wind_speed_max'], response_info['data']['measurements']['wind_speed_min'], response_info['data']['status']['state']])

In [117]:
data_mesures

[['2021-03-11T11:25:47.000Z', 113, None, 225, 31, None, None, 'on'],
 ['2021-03-11T11:20:59.000Z', 307, None, 247.5, 20.5, 31.5, 11, 'on'],
 ['2021-03-11T11:14:54.000Z', 308, None, 90, 0, 0, 0, 'on']]